In [3]:
from google.colab import files
uploaded = files.upload()

Saving APAN5400_Group-data_acquisition.zip to APAN5400_Group-data_acquisition.zip


In [6]:
!unzip APAN5400_Group-data_acquisition.zip

Archive:  APAN5400_Group-data_acquisition.zip
35725e5e143ff726334c46934f337d091979adbd
   creating: APAN5400_Group-data_acquisition/
  inflating: APAN5400_Group-data_acquisition/Data_Dictionary_v1.xlsx  
  inflating: APAN5400_Group-data_acquisition/README.md  
 extracting: APAN5400_Group-data_acquisition/Spotify  
  inflating: APAN5400_Group-data_acquisition/Spotify_data.ipynb  
  inflating: APAN5400_Group-data_acquisition/Youtube.ipynb  
   creating: APAN5400_Group-data_acquisition/kworb/
  inflating: APAN5400_Group-data_acquisition/kworb/Kworb.ipynb  
  inflating: APAN5400_Group-data_acquisition/kworb/README.md  
  inflating: APAN5400_Group-data_acquisition/kworb/spotify_artist_top10_ablums_20251120.csv  
  inflating: APAN5400_Group-data_acquisition/kworb/spotify_artist_top10_albums_20251120.json  
  inflating: APAN5400_Group-data_acquisition/kworb/spotify_monthly_listeners_20251120.csv  
  inflating: APAN5400_Group-data_acquisition/kworb/spotify_monthly_listeners_20251120.json  
  i

In [7]:
!ls

APAN5400_Group-data_acquisition      sample_data
APAN5400_Group-data_acquisition.zip


In [10]:
!ls APAN5400_Group-data_acquisition

Data_Dictionary_v1.xlsx		     Spotify_data.ipynb
kworb				     spotify_tracks_20251113_043306.csv
README.md			     spotify_tracks_20251113_043306.json
spotify				     youtube_channel_20251113_160003.json
Spotify				     Youtube.ipynb
spotify_artist_20251113_043306.json  youtube_videos_20251113_160003.csv


In [11]:
!ls APAN5400_Group-data_acquisition/spotify

README.MD		       spotify_tracks_20251120.csv
spotify_artists_20251120.csv   spotify_tracks_20251120.json
spotify_artists_20251120.json  Spotify_v2.ipynb


In [13]:
import os
os.listdir("APAN5400_Group-data_acquisition")

['youtube_channel_20251113_160003.json',
 'Spotify',
 'Youtube.ipynb',
 'spotify_tracks_20251113_043306.json',
 'Data_Dictionary_v1.xlsx',
 'spotify_artist_20251113_043306.json',
 'kworb',
 'Spotify_data.ipynb',
 'spotify_tracks_20251113_043306.csv',
 'README.md',
 'spotify',
 'youtube_videos_20251113_160003.csv']

In [16]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("PopFlowAnalytics").getOrCreate()

tracks = spark.read.csv(
    "APAN5400_Group-data_acquisition/spotify/spotify_tracks_20251120.csv",
    header=True, inferSchema=True
)

artists = spark.read.csv(
    "APAN5400_Group-data_acquisition/spotify/spotify_artists_20251120.csv",
    header=True, inferSchema=True
)

us_daily = spark.read.csv(
    "APAN5400_Group-data_acquisition/kworb/spotify_us_daily_artist_20251120.csv",
    header=True, inferSchema=True
)

monthly = spark.read.csv(
    "APAN5400_Group-data_acquisition/kworb/spotify_monthly_listeners_20251120.csv",
    header=True, inferSchema=True
)

In [17]:
# Helper function: remove commas, cast to int
def clean_int(col):
    return F.regexp_replace(col, ",", "").cast("int")

# Clean us_daily
us_daily = (
    us_daily
    .withColumn("daily_streams_int", clean_int("daily_streams"))
    .withColumn("seven_day_int",    clean_int("seven_day"))
    .withColumn("total_streams_int",clean_int("total_streams"))
)

# Clean monthly
monthly = (
    monthly
    .withColumn("listeners_int", clean_int("listeners"))
)

Daily Playback Trend

In [18]:
import datetime
from pyspark.sql.types import StringType

# Pick snapshot date (based on filename 20251120)
snapshot_date = datetime.date(2025, 11, 20)

# Generate last 7 days
date_list = [(snapshot_date - datetime.timedelta(days=i)).isoformat()
             for i in range(6, -1, -1)]

date_list

['2025-11-14',
 '2025-11-15',
 '2025-11-16',
 '2025-11-17',
 '2025-11-18',
 '2025-11-19',
 '2025-11-20']

In [19]:
# Create an array column with 7 dates
daily_playback = (
    us_daily
    .withColumn("date_array", F.array([F.lit(d) for d in date_list]))
    .withColumn("stream_per_day", (F.col("seven_day_int") / 7))
    .withColumn("singer_name", F.col("artist_name"))
    .select("singer_name", "date_array", "stream_per_day")
    .withColumn("date", F.explode("date_array"))
    .withColumn("playback_date", F.col("date"))
    .withColumn("playback_count", F.col("stream_per_day").cast("int"))
    .select("singer_name", "playback_date", "playback_count")
)

daily_playback.show(10)

+------------+-------------+--------------+
| singer_name|playback_date|playback_count|
+------------+-------------+--------------+
|Taylor Swift|   2025-11-14|       1435447|
|Taylor Swift|   2025-11-15|       1435447|
|Taylor Swift|   2025-11-16|       1435447|
|Taylor Swift|   2025-11-17|       1435447|
|Taylor Swift|   2025-11-18|       1435447|
|Taylor Swift|   2025-11-19|       1435447|
|Taylor Swift|   2025-11-20|       1435447|
|     HUNTR/X|   2025-11-14|       1317997|
|     HUNTR/X|   2025-11-15|       1317997|
|     HUNTR/X|   2025-11-16|       1317997|
+------------+-------------+--------------+
only showing top 10 rows



In [20]:
daily_playback.coalesce(1).write.mode("overwrite").csv(
    "frontend_daily_playback",
    header=True
)

Song Popularity

In [22]:
tracks.printSchema()
tracks.show(5)

root
 |-- track_id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- album_id: string (nullable = true)

+--------------------+--------------------+----------+--------------------+
|            track_id|          track_name|popularity|            album_id|
+--------------------+--------------------+----------+--------------------+
|53iuhJlwXhSER5J2I...| The Fate of Ophelia|       100|4a6NzYL1YHRUgx9e3...|
|1CPZ5BxNNd0n0nF4O...|              Golden|        99|14JkAa6IiFaOh5s0n...|
|1qbmS6ep2hbBRaEZF...|          Man I Need|        96|0Bv6OJO0L5gcf20DM...|
|6sGIMrtIzQjdzNndV...|So Easy (To Fall ...|        93|0l8zYqoUeBYg47Gme...|
|3yWuTOYDztXjZxdE2...|             Opalite|        96|4a6NzYL1YHRUgx9e3...|
+--------------------+--------------------+----------+--------------------+
only showing top 5 rows



In [23]:
artists.printSchema()
artists.show(5)

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- followers: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- top_tracks: string (nullable = true)

+--------------------+------------+----------+---------+--------+--------------------+
|           artist_id| artist_name|popularity|followers|  genres|          top_tracks|
+--------------------+------------+----------+---------+--------+--------------------+
|06HL4z0CvFAxyc27G...|Taylor Swift|       100|146488353|    NULL|['53iuhJlwXhSER5J...|
|2yNNYQBChuox9A5Ka...|     HUNTR/X|        87|  3868352|   k-pop|['1CPZ5BxNNd0n0nF...|
|00x1fYSGhdqScXBRp...| Olivia Dean|        90|  1836924|pop soul|['1qbmS6ep2hbBRaE...|
|0fTSzq9jAh4c36UVb...| Alex Warren|        85|  3002574|    NULL|['2RkZ5LkEzeHGRsm...|
|4G9NDjRyZFDlJKMRL...|       sombr|        86|  2948547|    NULL|['7qjZnBKE73H4Oxk...|
+--------------------+------------+----------+---

In [24]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# 1) Clean and split top_tracks into an array of track_ids
artists_tracks = (
    artists
    # remove [ ] ' characters
    .withColumn("top_tracks_clean",
                F.regexp_replace("top_tracks", r"[\[\]']", ""))
    # split by comma and optional space -> array of track_ids
    .withColumn("top_tracks_array",
                F.split(F.col("top_tracks_clean"), r",\s*"))
    # explode so each row = one (artist, track_id)
    .withColumn("track_id", F.explode("top_tracks_array"))
    .select("artist_id", "artist_name", "track_id")
)

artists_tracks.show(5)

+--------------------+------------+--------------------+
|           artist_id| artist_name|            track_id|
+--------------------+------------+--------------------+
|06HL4z0CvFAxyc27G...|Taylor Swift|53iuhJlwXhSER5J2I...|
|06HL4z0CvFAxyc27G...|Taylor Swift|3yWuTOYDztXjZxdE2...|
|06HL4z0CvFAxyc27G...|Taylor Swift|1jgTiNob5cVyXeJ3W...|
|06HL4z0CvFAxyc27G...|Taylor Swift|03bTIHJElXZ0O0jqO...|
|06HL4z0CvFAxyc27G...|Taylor Swift|5eXgqtg3T8Av0m1FU...|
+--------------------+------------+--------------------+
only showing top 5 rows



In [25]:
tracks_with_artist = tracks.join(
    artists_tracks,
    on="track_id",
    how="left"
)

tracks_with_artist.select(
    "artist_name", "track_name", "popularity"
).show(10, truncate=False)

+-----------------+---------------------------------+----------+
|artist_name      |track_name                       |popularity|
+-----------------+---------------------------------+----------+
|Taylor Swift     |The Fate of Ophelia              |100       |
|HUNTR/X          |Golden                           |99        |
|Olivia Dean      |Man I Need                       |96        |
|Olivia Dean      |So Easy (To Fall In Love)        |93        |
|Taylor Swift     |Opalite                          |96        |
|Alex Warren      |Ordinary                         |91        |
|NULL             |back to friends                  |92        |
|The Neighbourhood|Sweater Weather                  |92        |
|The Goo Goo Dolls|Iris                             |93        |
|Brenda Lee       |Rockin' Around The Christmas Tree|85        |
+-----------------+---------------------------------+----------+
only showing top 10 rows



In [26]:
song_popularity = (
    tracks_with_artist
    .select(
        F.col("artist_name").alias("singer_name"),
        F.col("track_name"),
        F.col("popularity").alias("popularity_score")
    )
    .dropna(subset=["singer_name"])  # keep only tracks we could match to an artist
)

# Rank top 5 tracks within each artist (optional)
w_artist = Window.partitionBy("singer_name").orderBy(F.col("popularity_score").desc())

song_popularity_ranked = (
    song_popularity
    .withColumn("rank_within_artist", F.dense_rank().over(w_artist))
    .filter(F.col("rank_within_artist") <= 5)
)

song_popularity_ranked.show(20, truncate=False)

# Save for front-end
song_popularity_ranked.coalesce(1).write.mode("overwrite").csv(
    "frontend_song_popularity",
    header=True
)

+--------------+-----------------------------------------------------------------------------------+----------------+------------------+
|singer_name   |track_name                                                                         |popularity_score|rank_within_artist|
+--------------+-----------------------------------------------------------------------------------+----------------+------------------+
|A$AP Rocky    |I Smoked Away My Brain (I'm God x Demons Mashup) (feat. Imogen Heap & Clams Casino)|85              |1                 |
|Alex Warren   |Ordinary                                                                           |91              |1                 |
|Andy Williams |It's the Most Wonderful Time of the Year                                           |72              |1                 |
|Arctic Monkeys|I Wanna Be Yours                                                                   |91              |1                 |
|Arctic Monkeys|505                      

In [27]:
tracks.printSchema()
artists.printSchema()
us_daily.printSchema()
monthly.printSchema()

root
 |-- track_id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- album_id: string (nullable = true)

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- followers: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- top_tracks: string (nullable = true)

root
 |-- rank: integer (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- daily_streams: string (nullable = true)
 |-- streams_plus: string (nullable = true)
 |-- seven_day: string (nullable = true)
 |-- seven_day_plus: string (nullable = true)
 |-- total_streams: string (nullable = true)
 |-- artist_url: string (nullable = true)
 |-- track_url: string (nullable = true)
 |-- daily_streams_int: integer (nullable = true)
 |-- seven_day_int: integer (nullable = true)
 |-- total_streams_int: integer (nullable = tru

**Song Popularity Table (for bar chart)

In [28]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

artists_tracks = (
    artists
    .withColumn("top_tracks_clean", F.regexp_replace("top_tracks", r"[\[\]']", ""))
    .withColumn("top_tracks_array", F.split(F.col("top_tracks_clean"), r",\s*"))
    .withColumn("track_id", F.explode("top_tracks_array"))
    .select("artist_id", "artist_name", "track_id")
)

In [29]:
tracks_with_artist = tracks.join(
    artists_tracks,
    on="track_id",
    how="left"
)

In [30]:
song_popularity = (
    tracks_with_artist
    .select(
        F.col("artist_name").alias("singer_name"),
        "track_name",
        F.col("popularity").alias("popularity_score")
    )
    .dropna(subset=["singer_name"])
)

In [31]:
w = Window.partitionBy("singer_name").orderBy(F.desc("popularity_score"))

song_popularity_ranked = (
    song_popularity
    .withColumn("rank_within_artist", F.dense_rank().over(w))
    .filter(F.col("rank_within_artist") <= 5)
)

In [32]:
song_popularity_ranked.coalesce(1).write.mode("overwrite").csv(
    "frontend_song_popularity",
    header=True
)

In [33]:
song_popularity_ranked.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("frontend_song_popularity")

In [34]:
import os
os.listdir("frontend_song_popularity")

['._SUCCESS.crc',
 '.part-00000-a27e6e2a-edd8-4004-8bd8-966070b998c0-c000.csv.crc',
 '_SUCCESS',
 'part-00000-a27e6e2a-edd8-4004-8bd8-966070b998c0-c000.csv']

In [35]:
import shutil
import glob

# Find the part file
part_file = glob.glob("frontend_song_popularity/part*.csv")[0]

# Rename to a nice file
shutil.move(part_file, "frontend_song_popularity.csv")

'frontend_song_popularity.csv'

In [38]:
from google.colab import files
files.download("frontend_song_popularity.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Daily Playback Table (for line chart)

In [39]:
def clean_int(col):
    return F.regexp_replace(col, ",", "").cast("int")

us_daily_clean = (
    us_daily
    .withColumn("daily_streams_int", clean_int("daily_streams"))
    .withColumn("seven_day_int", clean_int("seven_day"))
)

In [40]:
import datetime

snapshot = datetime.date(2025, 11, 20)   # from your file name
dates = [(snapshot - datetime.timedelta(days=i)).isoformat()
         for i in range(6, -1, -1)]

In [41]:
daily_playback = (
    us_daily_clean
    .withColumn("date_array", F.array([F.lit(d) for d in dates]))
    .withColumn("stream_per_day", (F.col("seven_day_int") / 7))
    .withColumn("singer_name", F.col("artist_name"))
    .select("singer_name", "date_array", "stream_per_day")
    .withColumn("playback_date", F.explode("date_array"))
    .withColumn("playback_count", F.col("stream_per_day").cast("int"))
    .select("singer_name", "playback_date", "playback_count")
)

In [44]:
daily_playback.coalesce(1).write.mode("overwrite").csv(
    "frontend_daily_playback",
    header=True
)

In [45]:
import glob

glob.glob("frontend_daily_playback/*.csv")

['frontend_daily_playback/part-00000-a98ca81e-5b84-430f-831b-cc3722af9f2e-c000.csv']

In [46]:
import shutil
import glob

part_file = glob.glob("frontend_daily_playback/part*.csv")[0]
shutil.copy(part_file, "frontend_daily_playback.csv")
"Saved as frontend_daily_playback.csv"

'Saved as frontend_daily_playback.csv'

In [47]:
from google.colab import files
files.download("frontend_daily_playback.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**US Map Table (top song per state)

In [48]:
top_song = us_daily_clean.orderBy("rank").limit(1).first()

In [49]:
states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA",
          "HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
          "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ",
          "NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
          "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]

states_df = spark.createDataFrame([(s,) for s in states], ["state"])

In [50]:
us_map = (
    states_df
    .withColumn("artist_name", F.lit(top_song["artist_name"]))
    .withColumn("track_name", F.lit(top_song["track_name"]))
    .withColumn("streams", F.lit(top_song["daily_streams_int"]))
)

In [51]:
us_map.coalesce(1).write.mode("overwrite").csv(
    "frontend_us_map",
    header=True
)

In [53]:
import glob

glob.glob("frontend_us_map/*.csv")

['frontend_us_map/part-00000-9976199a-ec58-4442-9134-7f03c0529c0c-c000.csv']

In [54]:
import shutil
import glob

part_file = glob.glob("frontend_us_map/part*.csv")[0]
shutil.copy(part_file, "frontend_us_map.csv")
"Saved as frontend_us_map.csv"

'Saved as frontend_us_map.csv'

In [55]:
from google.colab import files
files.download("frontend_us_map.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>